In [3]:
import tensorflow as tf
from google.cloud import bigquery
import datetime
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from math import sqrt
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from datetime import datetime


In [4]:
# create window
def df_to_X(df, window_size):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [r for r in df_as_np[i:i+window_size]]
    X.append(row)
    
  return np.array(X)

In [5]:
# load model
model = tf.keras.models.load_model('heath.h5')

In [6]:
# project details and gcp connection
project_id = "galvanic-smoke-394310"
key_path = r'C:\Users\gabri\GCP\galvanic-smoke-394310-c595ae82c676_bigquery.json'
client = bigquery.Client.from_service_account_json(key_path)

In [7]:
# define data windows
input_days = 2
input_hours = input_days * 24
prediction_window = 24
window_size = input_hours - prediction_window

current_time = datetime.now().replace(minute=0, second=0, microsecond=0)
print(f'Window starts {window_size} hours before {current_time}')

Window starts 24 hours before 2023-08-19 17:00:00


In [8]:
# get data query
query = f"""
SELECT 
  lon,
  lat,
  localtime_epoch,
  name,
  pressure,
  temperature,
  wind_speed,
  wind_degree,
  humidity
FROM `galvanic-smoke-394310.weather_prediction.current_weather_current` as cwc
JOIN `galvanic-smoke-394310.weather_prediction.current_weather_location` as cwl
ON cwc._airbyte_current_weather_hashid = cwl._airbyte_current_weather_hashid
where name = 'Stanwell'
order by localtime_epoch desc
limit {input_hours} 
"""
query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()
df['localtime_datetime'] = pd.to_datetime(df['localtime_epoch'], unit='s')
y_true = df.iloc[0:prediction_window]
df.set_index('localtime_datetime', inplace=True)
df.head(2)

,lon,lat,localtime_epoch,name,pressure,temperature,wind_speed,wind_degree,humidity
localtime_datetime,,,,,,,,,
2023-08-19 17:17:00,-0.483,51.451,1.692465e+09,Stanwell,1016.0,23.0,28.0,220.0,57.0
2023-08-19 16:17:00,-0.483,51.451,1.692462e+09,Stanwell,1016.0,23.0,30.0,220.0,57.0


In [9]:
temperature_df = pd.DataFrame({'temperature': df['temperature']})

X_pred = df_to_X(temperature_df, window_size=window_size)

y_predict = model.predict(x=X_pred)
y_predict = y_predict.reshape(len(y_predict), )

1/1 [==============================] - 1s 960ms/step


In [10]:
fig = go.Figure() 
# fig.add_trace(go.Scatter(x=[i for i in range(len(y_predict))], y=y_predict))
fig.add_trace(go.Scatter(x=df.index, y=df['temperature'].to_list()))


In [26]:
current_utc_time = datetime.utcnow()
formatted_utc_time = current_utc_time.strftime("%Y-%m-%d %H:%M")

london_tz = pytz.timezone('Europe/London')
current_london_time = datetime.now(london_tz)
formatted_london_time = current_london_time.strftime("%Y-%m-%d %H:%M")

print(f'Last update at: ({formatted_utc_time} UTC) and ({formatted_london_time} London Time)')

Last update at: (2023-08-19 16:37 UTC) and (2023-08-19 17:37 London Time)


In [17]:
london_tz = pytz.timezone('Europe/London')
current_london_time = datetime.now(london_tz)
print(current_london_time)

2023-08-19 17:34:39.449450+01:00


In [11]:
fig = go.Figure() 
fig.add_trace(go.Scatter(x=[i for i in range(len(y_predict))], y=y_predict))